In [ ]:
pip install konlpy

In [9]:
import pandas as pd
from transformers import pipeline
from konlpy.tag import Okt

# 1. 데이터 로드
df = pd.read_csv("/content/경북산불_filtered2.csv", encoding="utf-8-sig")
df['content'] = df['content'].fillna('').astype(str)

# 2. 사용자 사전
custom_lexicon = ['산불','진화','헬기','소방관','소방서','소방청','대피','피해','화재','강풍','건조','기상청','예보','경보','주의보','경계령','기상','산림청','산림문화유산','연기','불씨','불길','전소','이재민','주택마을','마을회관','비상복구지원금','복구비예산지원','의용소방대','초기대응','대응출동','인명피해','사망자','부상자','실종자','대피소','임시거처','현장','전력','전기','통신','차단','소화기','산림','보호','불법','소각','입산통제원','인조사담당자','경북','경상북도','의성','봉화','안동','영덕','영양','청송','강원','강원도','강릉','동해','속초','울진','기부','자원봉사','복구작업','수색','긴급대책정부','행안부','지자체','소방대','소방장비','방화열','감지기','드론','산불','감시원','지휘본부','현장통제','언론보도','방송전파','구호모금후원','공무원','차량','도로','교통','산사태','낙석피난','탈출','불안대책본부','예방','예방활동캠페인','산불예방재난','재난문자','재난방송','행정안전부','소방청장','사고화재현장','잔불','열기','폭염','기온','건조','특보','진화작업','진화율','소실면적농가','축사폐기물','현장조사','방문현황실태','상황실비상근무','소방본부','구호물품','보급담요','생수','방재','물자','전국인력','의료진','긴급','구호','산림피해','조사단','경찰군부대','출입','통제','방범','경계','방화범','조사처벌','형사처벌','과태료위반','행위주의','기초자치단체','중앙정부','시군구피해신고','구호센터지자체장','현장','지도산불진화대','현장대응','기상상황특보','바람','습도','햇볕','산불진원','지산불','상황실']


# 3. 형태소 분석기 초기화
okt = Okt()

# 4. 모델 설정
model_identifier = "nlptown/bert-base-multilingual-uncased-sentiment"

# 5. 파이프라인 초기화
sentiment_analyzer = pipeline(
    "text-classification",
    model=model_identifier,
    tokenizer=model_identifier  # 토크나이저도 동일 모델 지정
)

# 6. 감성 분류 함수
def hybrid_sentiment_analysis(text):
    # 1단계: 사용자 사전 단어 탐지
    tokens = okt.morphs(text)
    detected_terms = [word for word in tokens if word in custom_lexicon]

    if not detected_terms:
        return '중립'

    # 2단계: 감성 분석 수행
    try:
        result = sentiment_analyzer(text[:512])[0]
        label = result['label']

        # 레이블 변환 로직 (수정된 부분)
        label_num = int(label.split()[0])  # '1 star' → 1
        if label_num >= 4:
            return '긍정'
        elif label_num <= 2:
            return '부정'
        else:
            return '중립'

    except Exception as e:
        print(f"분석 실패: {str(e)[:50]}...")  # 에러 메시지 축약 출력
        return '중립'

# 7. 감성 분석 실행
df['sentiment'] = df['content'].apply(hybrid_sentiment_analysis)

# 8. 결과 저장 및 확인
df.to_csv("custom_lexicon_sentiment.csv", index=False, encoding="utf-8-sig")
print(df['sentiment'].value_counts())



config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


sentiment
부정    2567
긍정     365
중립      68
Name: count, dtype: int64


In [11]:
file_path = r"/content/custom_lexicon_sentiment.csv"
data = pd.read_csv(file_path)
data.head()
df['sentiment'].value_counts()

,count
sentiment,
부정,2567
긍정,365
중립,68
